In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('features.py'), '..')))

from NBAData.features import *

/Users/alexg/Documents/Documents/Prize-Picks-Prop-Predictor/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
pd.set_option('display.max_columns', None)

s21_playoffs = pd.read_csv('../CSV_FILES/PLAYOFF_DATA/PLAYOFFS_21_pts_FEATURES.csv')
s21_playoffs = assign_playoff_series_info(s21_playoffs)
s21_playoffs['IS_PLAYOFF'] = 1
s21_regular = pd.read_csv('../CSV_FILES/REGULAR_DATA/SEASON_21_pts_FEATURES.csv')
s21_regular['IS_PLAYOFF'] = 0
s21 = pd.concat([s21_playoffs, s21_regular])
s21['HEIGHT_IN_INCHES'] = s21['HEIGHT'].apply(convert_height_to_inches)
s21['GAME_DATE'] = pd.to_datetime(s21['GAME_DATE'])

s22_playoffs = pd.read_csv('../CSV_FILES/PLAYOFF_DATA/PLAYOFFS_22_pts_FEATURES.csv')
s22_playoffs = assign_playoff_series_info(s22_playoffs)
s22_playoffs['IS_PLAYOFF'] = 1
s22_regular = pd.read_csv('../CSV_FILES/REGULAR_DATA/SEASON_22_pts_FEATURES.csv')
s22_regular['IS_PLAYOFF'] = 0
s22 = pd.concat([s22_playoffs, s22_regular])
s22['HEIGHT_IN_INCHES'] = s22['HEIGHT'].apply(convert_height_to_inches) 
s22['GAME_DATE'] = pd.to_datetime(s22['GAME_DATE'])

s23_playoffs = pd.read_csv('../CSV_FILES/PLAYOFF_DATA/PLAYOFFS_23_pts_FEATURES.csv')  
s23_playoffs = assign_playoff_series_info(s23_playoffs)
s23_playoffs['IS_PLAYOFF'] = 1
s23_regular = pd.read_csv('../CSV_FILES/REGULAR_DATA/SEASON_23_pts_FEATURES.csv')
s23_regular['IS_PLAYOFF'] = 0
s23 = pd.concat([s23_playoffs, s23_regular])
s23['HEIGHT_IN_INCHES'] = s23['HEIGHT'].apply(convert_height_to_inches)
s23['GAME_DATE'] = pd.to_datetime(s23['GAME_DATE'])

s24_playoffs = pd.read_csv('../CSV_FILES/PLAYOFF_DATA/PLAYOFFS_24_pts_FEATURES.csv')
s24_playoffs = assign_playoff_series_info(s24_playoffs)
s24_playoffs['IS_PLAYOFF'] = 1
s24_regular = pd.read_csv('../CSV_FILES/REGULAR_DATA/SEASON_24_pts_FEATURES.csv')
s24_regular['IS_PLAYOFF'] = 0
s24 = pd.concat([s24_playoffs, s24_regular])
s24['HEIGHT_IN_INCHES'] = s24['HEIGHT'].apply(convert_height_to_inches)
s24['GAME_DATE'] = pd.to_datetime(s24['GAME_DATE'])

# s25_playoffs = pd.read_csv('CSV_FILES/PLAYOFF_DATA/PLAYOFFS_25_ast_FEATURES.csv')
# s25_playoffs = assign_playoff_series_info(s25_playoffs)
# s25_playoffs['IS_PLAYOFF'] = 1
# s25_regular = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_25_ast_FEATURES.csv')
# s25_regular['IS_PLAYOFF'] = 0
# s25 = pd.concat([s25_playoffs, s25_regular])
# s25['GAME_DATE'] = pd.to_datetime(s25['GAME_DATE'])

df = pd.concat([s21,s22,s23,s24])
df = df.drop(columns=['HEIGHT', 'Unnamed: 0', 'Unnamed: 0.1'])
df = df.sort_values(by='GAME_DATE', ascending=False).reset_index(drop=True)
df

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,COMMENT,OFF_RATING,E_OFF_RATING,DEF_RATING,E_DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,EFG_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,POSS,PACE_PER40,E_USG_PCT,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE,STARTING,MIN_ROLL_AVG_2,PTS_ROLL_AVG_2,PTS_STD_AVG_2,FGA_ROLL_AVG_2,FGM_ROLL_AVG_2,FG_PCT_ROLL_AVG_2,FG3A_ROLL_AVG_2,FG3M_ROLL_AVG_2,FG3_PCT_ROLL_AVG_2,FTM_ROLL_AVG_2,FTA_ROLL_AVG_2,FT_PCT_ROLL_AVG_2,USG_PCT_ROLL_AVG_2,TS_PCT_ROLL_AVG_2,EFG_PCT_ROLL_AVG_2,OREB_ROLL_AVG_2,DREB_ROLL_AVG_2,REB_ROLL_AVG_2,PLUS_MINUS_ROLL_AVG_2,PIE_ROLL_AVG_2,TEAM_FGA_ROLL_AVG_2,TEAM_FG_PCT_ROLL_AVG_2,TEAM_FG3A_ROLL_AVG_2,TEAM_FG3_PCT_ROLL_AVG_2,TEAM_FTM_ROLL_AVG_2,TEAM_FTA_ROLL_AVG_2,TEAM_FT_PCT_ROLL_AVG_2,TEAM_PTS_ROLL_AVG_2,TEAM_PACE_ROLL_AVG_2,TEAM_OFF_RATING_ROLL_AVG_2,OPP_DEF_RATING_ROLL_AVG_2,OPP_PACE_ROLL_AVG_2,OPP_FG_PCT_ROLL_AVG_2,MIN_ROLL_AVG_4,PTS_ROLL_AVG_4,PTS_STD_AVG_4,FGA_ROLL_AVG_4,FGM_ROLL_AVG_4,FG_PCT_ROLL_AVG_4,FG3A_ROLL_AVG_4,FG3M_ROLL_AVG_4,FG3_PCT_ROLL_AVG_4,FTM_ROLL_AVG_4,FTA_ROLL_AVG_4,FT_PCT_ROLL_AVG_4,USG_PCT_ROLL_AVG_4,TS_PCT_ROLL_AVG_4,EFG_PCT_ROLL_AVG_4,OREB_ROLL_AVG_4,DREB_ROLL_AVG_4,REB_ROLL_AVG_4,PLUS_MINUS_ROLL_AVG_4,PIE_ROLL_AVG_4,TEAM_FGA_ROLL_AVG_4,TEAM_FG_PCT_ROLL_AVG_4,TEAM_FG3A_ROLL_AVG_4,TEAM_FG3_PCT_ROLL_AVG_4,TEAM_FTM_ROLL_AVG_4,TEAM_FTA_ROLL_AVG_4,TEAM_FT_PCT_ROLL_AVG_4,TEAM_PTS_ROLL_AVG_4,TEAM_PACE_ROLL_AVG_4,TEAM_OFF_RATING_ROLL_AVG_4,OPP_DEF_RATING_ROLL_AVG_4,OPP_PACE_ROLL_AVG_4,OPP_FG_PCT_ROLL_AVG_4,MIN_ROLL_AVG_6,PTS_ROLL_AVG_6,PTS_STD_AVG_6,FGA_ROLL_AVG_6,FGM_ROLL_AVG_6,FG_PCT_ROLL_AVG_6,FG3A_ROLL_AVG_6,FG3M_ROLL_AVG_6,FG3_PCT_ROLL_AVG_6,FTM_ROLL_AVG_6,FTA_ROLL_AVG_6,FT_PCT_ROLL_AVG_6,USG_PCT_ROLL_AVG_6,TS_PCT_ROLL_AVG_6,EFG_PCT_ROLL_AVG_6,OREB_ROLL_AVG_6,DREB_ROLL_AVG_6,REB_ROLL_AVG_6,PLUS_MINUS_ROLL_AVG_6,PIE_ROLL_AVG_6,TEAM_FGA_ROLL_AVG_6,TEAM_FG_PCT_ROLL_AVG_6,TEAM_FG3A_ROLL_AVG_6,TEAM_FG3_PCT_ROLL_AVG_6,TEAM_FTM_ROLL_AVG_6,TEAM_FTA_ROLL_AVG_6,TEAM_FT_PCT_ROLL_AVG_6,TEAM_PTS_ROLL_AVG_6,TEAM_PACE_ROLL_AVG_6,TEAM_OFF_RATING_ROLL_AVG_6,OPP_DEF_RATING_ROLL_AVG_6,OPP_PACE_ROLL_AVG_6,OPP_FG_PCT_ROLL_AVG_6,PTS_LAG_1,PTS_LAG_2,PTS_LAG_3,PTS_LAG_4,PLAYER_HOME_AVG_PTS,PLAYER_AWAY_AVG_PTS,MATCHUP_AVG_PTS_LAST_3,GAMES_VS_OPP,DAYS_OF_REST,DAYS_AGO,OPP_ROLL_OPP_DEF_RATING_5,OPP_ROLL_OPP_STL_5,OPP_ROLL_OPP_BLK_5,NORMALIZED_DEF_RATING,OPP_DEF_STRENGTH,DEF_CATEGORY,AST_VS_DEF,AST_PCT_VS_DEF,AST_TOV_VS_DEF,USG_PCT_VS_DEF,PACE_VS_DEF,POSS_VS_DEF,OFF_RATING_VS_DEF,WEIGHT,GUARD,FORWARD,CENTER,MATCHUP_KEY,Series,GameInSeries,SeriesName,IS_PLAYOFF,PTS_VS_DEF,FGA_VS_DEF,FTA_VS_DEF,FG3A_VS_DEF,HEIGHT_IN_INCHES
0,Derrick White,1628401,BOS vs. DAL,BOS,1610612738,DAL,1,42300405,2024-06-17,W,38,14,1,8,4,10,0.400,4,8,0.500,2,2,1.000,4,4,2,1,2,0,1,32.1,1.287,0.600000,G,NaN,111.4,105.5,106.9,104.4,4.5,0.089,0.121,0.103,0.043,0.600,0.50,0.149,0.643,92.92,89.37,0.126,70.0,74.47,0.150,"JUN 17, 2024",240,38,89,0.427,13,39,0.333,17,20,0.850,15,36,51,25,9,2,7,15,106,118.040089,4.0,4.0,35.0,0.448718,1.610613e+09,118.040089,89.80,89.76,89.72,1,36.5,11.0,7.07,9.0,3.0,0.33,8.5,3.0,0.35,2.0,2.0,1.00,0.14,0.53,0.49,0.0,4.5,4.5,-13.0,0.06,81.0,0.41,43.5,0.36,12.5,13.5,0.93,95.0,92.94,102.48,102.48,94.86,0.47,36.50,13.75,5.32,11.00,4.25,0.38,8.75,3.25,0.37,2.00,2.00,1.00,0.17,0.56,0.52,0.75,3.25,4.00,-3.75,0.08,82.00,0.44,42.00,0.34,14.25,16.50,0.87,100.50,92.76,108.49,108.49,94.91,0.46,37.17,14.00,4.24,11.50,4.50,0.38,8.67,2.83,0.32,2.17,2.33,0.96,0.17,0.55,0.51,0.83,3.33,4.17,-1.00,0.09,83.33,0.45,43.00,0.34,14.67,17.67,0.

### Features that will be used to train my XGBoost model

In [10]:
features = [
    'MIN','FGA', 'FTA', 'FG3A','FG_PCT', 'FT_PCT', 'FG3_PCT', 'REB','OREB', 'DREB', 'STL', 'BLK', 'TOV', 'PF',
    'OFF_RATING','E_OFF_RATING', 'DEF_RATING', 'E_DEF_RATING', 'NET_RATING', 'PointsPerShot', 'EFG_PCT',
    'AST_PCT', 'AST_TOV','USG_PCT', 'TS_PCT','PACE', 'PIE', 'POSS', 'E_USG_PCT', 'PLUS_MINUS',
    'TEAM_FGA', 'TEAM_FG3A','TEAM_FG_PCT','TEAM_FG3_PCT','TEAM_AST', 'TEAM_REB', 'TEAM_STL', 'TEAM_BLK', 
    'TEAM_OFF_RATING', 'TEAM_PACE', 'TEAM_PTS',
    #grab oppenents avgerages for the game
    'OPP_PACE', 'OPP_DEF_RATING','OPP_STL', 'OPP_BLK', 'OPP_REB', 'OPP_FG_PCT',
    #rolling averages
    'MIN_ROLL_AVG_2', 'PTS_ROLL_AVG_2', 'PTS_STD_AVG_2', 'FGA_ROLL_AVG_2',
    'FGM_ROLL_AVG_2', 'FG_PCT_ROLL_AVG_2', 'FG3A_ROLL_AVG_2', 'FG3M_ROLL_AVG_2',
    'FG3_PCT_ROLL_AVG_2', 'FTM_ROLL_AVG_2', 'FTA_ROLL_AVG_2', 'FT_PCT_ROLL_AVG_2',
    'USG_PCT_ROLL_AVG_2', 'TS_PCT_ROLL_AVG_2', 'EFG_PCT_ROLL_AVG_2',
    'OREB_ROLL_AVG_2', 'DREB_ROLL_AVG_2', 'REB_ROLL_AVG_2',
    'PLUS_MINUS_ROLL_AVG_2', 'PIE_ROLL_AVG_2', 'TEAM_FGA_ROLL_AVG_2',
    'TEAM_FG_PCT_ROLL_AVG_2', 'TEAM_FG3A_ROLL_AVG_2', 'TEAM_FG3_PCT_ROLL_AVG_2',
    'TEAM_FTM_ROLL_AVG_2', 'TEAM_FTA_ROLL_AVG_2', 'TEAM_FT_PCT_ROLL_AVG_2',
    'TEAM_PTS_ROLL_AVG_2', 'TEAM_PACE_ROLL_AVG_2', 'TEAM_OFF_RATING_ROLL_AVG_2',
    'OPP_DEF_RATING_ROLL_AVG_2', 'OPP_PACE_ROLL_AVG_2', 'OPP_FG_PCT_ROLL_AVG_2',
    'MIN_ROLL_AVG_4', 'PTS_ROLL_AVG_4', 'PTS_STD_AVG_4', 'FGA_ROLL_AVG_4',
    'FGM_ROLL_AVG_4', 'FG_PCT_ROLL_AVG_4', 'FG3A_ROLL_AVG_4', 'FG3M_ROLL_AVG_4',
    'FG3_PCT_ROLL_AVG_4', 'FTM_ROLL_AVG_4', 'FTA_ROLL_AVG_4', 'FT_PCT_ROLL_AVG_4',
    'USG_PCT_ROLL_AVG_4', 'TS_PCT_ROLL_AVG_4', 'EFG_PCT_ROLL_AVG_4',
    'OREB_ROLL_AVG_4', 'DREB_ROLL_AVG_4', 'REB_ROLL_AVG_4',
    'PLUS_MINUS_ROLL_AVG_4', 'PIE_ROLL_AVG_4', 'TEAM_FGA_ROLL_AVG_4',
    'TEAM_FG_PCT_ROLL_AVG_4', 'TEAM_FG3A_ROLL_AVG_4', 'TEAM_FG3_PCT_ROLL_AVG_4',
    'TEAM_FTM_ROLL_AVG_4', 'TEAM_FTA_ROLL_AVG_4', 'TEAM_FT_PCT_ROLL_AVG_4',
    'TEAM_PTS_ROLL_AVG_4', 'TEAM_PACE_ROLL_AVG_4', 'TEAM_OFF_RATING_ROLL_AVG_4',
    'OPP_DEF_RATING_ROLL_AVG_4', 'OPP_PACE_ROLL_AVG_4', 'OPP_FG_PCT_ROLL_AVG_4',
    'MIN_ROLL_AVG_6', 'PTS_ROLL_AVG_6', 'PTS_STD_AVG_6', 'FGA_ROLL_AVG_6',
    'FGM_ROLL_AVG_6', 'FG_PCT_ROLL_AVG_6', 'FG3A_ROLL_AVG_6', 'FG3M_ROLL_AVG_6',
    'FG3_PCT_ROLL_AVG_6', 'FTM_ROLL_AVG_6', 'FTA_ROLL_AVG_6', 'FT_PCT_ROLL_AVG_6',
    'USG_PCT_ROLL_AVG_6', 'TS_PCT_ROLL_AVG_6', 'EFG_PCT_ROLL_AVG_6',
    'OREB_ROLL_AVG_6', 'DREB_ROLL_AVG_6', 'REB_ROLL_AVG_6',
    'PLUS_MINUS_ROLL_AVG_6', 'PIE_ROLL_AVG_6', 'TEAM_FGA_ROLL_AVG_6',
    'TEAM_FG_PCT_ROLL_AVG_6', 'TEAM_FG3A_ROLL_AVG_6', 'TEAM_FG3_PCT_ROLL_AVG_6',
    'TEAM_FTM_ROLL_AVG_6', 'TEAM_FTA_ROLL_AVG_6', 'TEAM_FT_PCT_ROLL_AVG_6',
    'TEAM_PTS_ROLL_AVG_6', 'TEAM_PACE_ROLL_AVG_6', 'TEAM_OFF_RATING_ROLL_AVG_6',
    'OPP_DEF_RATING_ROLL_AVG_6', 'OPP_PACE_ROLL_AVG_6', 'OPP_FG_PCT_ROLL_AVG_6',
    'PTS_LAG_1', 'PTS_LAG_2', 'PTS_LAG_3', 'PTS_LAG_4',
    'PLAYER_HOME_AVG_PTS', 'PLAYER_AWAY_AVG_PTS', 'MATCHUP_AVG_PTS_LAST_3',
    #ill input these during prediction
    'HEIGHT_IN_INCHES', 'WEIGHT', 'GUARD', 'FORWARD', 'CENTER', 'STARTING', 'DAYS_OF_REST', 'HOME_GAME', 'IS_PLAYOFF', 'Series', 'GameInSeries'
]

In [6]:
corr_matrix = df.corr(numeric_only=True)
target = corr_matrix['PTS'].sort_values(ascending=False).tail(15)
target


CENTER                  -0.016946
TEAM_TOV_ROLL_AVG_4     -0.017352
OPP_STL                 -0.019941
PACE_VS_DEF             -0.020397
TEAM_TOV                -0.021500
TEAM_TOV_ROLL_AVG_6     -0.021592
PACE_PER40              -0.025217
PACE                    -0.025217
OPP_REB                 -0.044189
FORWARD                 -0.049408
DEF_CATEGORY            -0.067852
OPP_DEF_STRENGTH        -0.081163
NORMALIZED_DEF_RATING   -0.096458
DAYS_OF_REST            -0.108291
PLAYER_ID               -0.117449
Name: PTS, dtype: float64

## Train XGBoost model

In [11]:
from xgboost_model import *

X = df[features]
y = df['PTS']
modelPTS = train_xgb_model(X, y, stat_line='PTS')
modelPTS

Fitting 3 folds for each of 50 candidates, totalling 150 fits

Model Performance Metrics for PTS:
R2 Score: 0.9985
MAE: 0.2184
RMSE: 0.3414

Best Parameters: {'subsample': 0.8, 'reg_lambda': 5, 'reg_alpha': 0.5, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.8}
Model saved to Models/PTS_xgb_model.pkl


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [14]:
import shap

# Create SHAP explainer
explainer = shap.TreeExplainer(modelPTS)
shap_values = explainer.shap_values(X)

# Get feature importance scores
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': np.abs(shap_values).mean(axis=0)
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nTop 15 Most Important Features For XGBoost Model:")
print(feature_importance.head(15))



Top 15 Most Important Features For XGBoost Model:
                    feature  importance
1                       FGA    4.131930
152  MATCHUP_AVG_PTS_LAST_3    1.510335
19            PointsPerShot    1.420369
24                   TS_PCT    0.856166
2                       FTA    0.854084
4                    FG_PCT    0.228436
5                    FT_PCT    0.226120
20                  EFG_PCT    0.159320
26                      PIE    0.115455
6                   FG3_PCT    0.107435
3                      FG3A    0.067154
27                     POSS    0.033931
23                  USG_PCT    0.027683
150     PLAYER_HOME_AVG_PTS    0.025429
151     PLAYER_AWAY_AVG_PTS    0.025331


In [15]:
import shap

# Create SHAP explainer
explainer = shap.TreeExplainer(modelPTS)
shap_values = explainer.shap_values(X)

# Get feature importance scores
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': np.abs(shap_values).mean(axis=0)
})

less_important_features = feature_importance[feature_importance['importance'] < 0.001]
print("\nTop 15 Least Important Features For XGBoost Model:")
print(less_important_features.head(15))


Top 15 Least Important Features For XGBoost Model:
         feature  importance
8           OREB    0.000001
10           STL    0.000056
11           BLK    0.000007
12           TOV    0.000035
15  E_OFF_RATING    0.000305
18    NET_RATING    0.000091
25          PACE    0.000005
29    PLUS_MINUS    0.000000
30      TEAM_FGA    0.000112
31     TEAM_FG3A    0.000105
32   TEAM_FG_PCT    0.000088
33  TEAM_FG3_PCT    0.000073
34      TEAM_AST    0.000025
35      TEAM_REB    0.000021
36      TEAM_STL    0.000004
